<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#X_train-and-y_train" data-toc-modified-id="X_train-and-y_train-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>X_train and y_train</a></span></li><li><span><a href="#X_test" data-toc-modified-id="X_test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>X_test</a></span></li><li><span><a href="#LinearRegression" data-toc-modified-id="LinearRegression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>LinearRegression</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Random Forest</a></span><ul class="toc-item"><li><span><a href="#Random-Forest-with-n_estimators=1000" data-toc-modified-id="Random-Forest-with-n_estimators=1000-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Random Forest with n_estimators=1000</a></span></li><li><span><a href="#Random-Forest-with-n_estimators=3000" data-toc-modified-id="Random-Forest-with-n_estimators=3000-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Random Forest with n_estimators=3000</a></span></li></ul></li><li><span><a href="#CatBoost-Regressor" data-toc-modified-id="CatBoost-Regressor-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>CatBoost Regressor</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Apply-the-best-parameter-from-RandomizeCV-back-to-the-model" data-toc-modified-id="Apply-the-best-parameter-from-RandomizeCV-back-to-the-model-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Apply the best parameter from RandomizeCV back to the model</a></span></li></ul></li></ul></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#Ensemble-model" data-toc-modified-id="Ensemble-model-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Ensemble model</a></span><ul class="toc-item"><li><span><a href="#Tune-Hyperparameters" data-toc-modified-id="Tune-Hyperparameters-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Tune Hyperparameters</a></span></li><li><span><a href="#Stacking" data-toc-modified-id="Stacking-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Stacking</a></span></li><li><span><a href="#Ensemble-result" data-toc-modified-id="Ensemble-result-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Ensemble result</a></span></li></ul></li><li><span><a href="#Stacked-model-(top-2-models)" data-toc-modified-id="Stacked-model-(top-2-models)-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Stacked model (top 2 models)</a></span></li><li><span><a href="#Ensemble-sample-testing" data-toc-modified-id="Ensemble-sample-testing-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Ensemble sample testing</a></span></li></ul></div>

In [8]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Load Data

In [11]:
# the cleaned version of df after removing outliners and adding new features
df = pd.read_pickle('./data/df_wip2')
prop = pd.read_pickle('./data/properties_2016_p')
sample = pd.read_pickle('./data/sample_p')

df_test = pd.read_pickle('./data/df_wip_test')

In [12]:
test_columns = df_test.columns
test_columns = test_columns.drop(['parcelid', '201610', '201611', '201612', '201710', '201711', '201712',
                                 'pooltypeid2', 'N_amenities', 'pooltypeid10',
                                  'taxdelinquencyflag', 'fireplaceflag', 'taxdelinquencyyear'])
test_columns

Index(['airconditioningtypeid', 'architecturalstyletypeid', 'basementsqft',
       'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet50', 'fips',
       'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid7', 'propertycountylandusecode',
       'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount',
       'censustractandblock', 'N_building_age', 'N_living_area_ratio',
       'N_structure_value_ratio', 'N_building_quality_room_cnt', 'N_location',
       'N_building_quality_location', 'N_county_

# X_train and y_train

In [13]:
from sklearn.preprocessing import StandardScaler

X = df[test_columns]
X_train = StandardScaler().fit_transform(X)

#log-transform the target
y_train = np.log1p(df['logerror'].values.astype(np.float32))

In [14]:
X_train.shape

(167046, 48)

# X_test

In [15]:
X_test = df_test[test_columns]
X_test = StandardScaler().fit_transform(X_test)

In [16]:
print(X_test.shape)
print(X_train.shape)
print(y_train.shape)

(2985217, 48)
(167046, 48)
(167046,)


# LinearRegression

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
lr.fit(X_train,y_train)
lr_y_pred = lr.predict(X_test)

In [10]:
for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = lr_y_pred
    
sample.to_csv('./data/lr_sample_p', index=False,float_format='%.5f')

# Random Forest

## Random Forest with n_estimators=1000

n_estimators=1000, max_features=5 score is 0.07857, 0.06743

n_estimators=1000, max_features=1 score is 0.07806, 0.06730

In [11]:
# Initialize the random forest regressor model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=1000, max_features=5, n_jobs=-1,random_state=42)

# fit model
rf.fit(X_train, y_train)

# prediction
rf_1000_y_pred = rf.predict(X_test)

In [12]:
for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = rf_1000_y_pred
    
sample.to_csv('./data/rf_n_estimators=1000_sample_p', index=False,float_format='%.5f')

## Random Forest with n_estimators=3000

In [13]:
# Initialize the random forest regressor model
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=3000, max_features=1, n_jobs=-1,random_state=42)

# fit model
rf.fit(X_train, y_train)

# prediction
rf_3000_y_pred = rf.predict(X_test)

In [14]:
for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = rf_3000_y_pred
    
sample.to_csv('./data/rf_n_estimators=3000_sample_p', index=False,float_format='%.5f')

In [15]:
rf_importances = rf.feature_importances_

In [16]:
feature_importances = pd.DataFrame(rf_importances,
                                   index = test_columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
N_property_tax_value_ratio,0.054626
taxamount,0.054371
structuretaxvaluedollarcnt,0.053954
taxvaluedollarcnt,0.053933
landtaxvaluedollarcnt,0.053318
N_structure_value_ratio,0.051924
N_living_area_ratio,0.051121
calculatedfinishedsquarefeet,0.050517
N_building_quality_location,0.049789
N_location,0.049637


# CatBoost Regressor

In [18]:
from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(iterations=1000,learning_rate=0.01, depth=9, loss_function='MAE', 
                          eval_metric='MAE', random_seed=1)
# fit model
model_cat.fit(X_train, y_train)

# prediction
model_cat_y_pred = model_cat.predict(X_test)

0:	learn: 0.0608328	total: 227ms	remaining: 3m 46s
1:	learn: 0.0607231	total: 351ms	remaining: 2m 55s
2:	learn: 0.0606256	total: 435ms	remaining: 2m 24s
3:	learn: 0.0605550	total: 555ms	remaining: 2m 18s
4:	learn: 0.0604884	total: 632ms	remaining: 2m 5s
5:	learn: 0.0604272	total: 705ms	remaining: 1m 56s
6:	learn: 0.0603785	total: 747ms	remaining: 1m 45s
7:	learn: 0.0603343	total: 794ms	remaining: 1m 38s
8:	learn: 0.0602903	total: 836ms	remaining: 1m 32s
9:	learn: 0.0602603	total: 880ms	remaining: 1m 27s
10:	learn: 0.0602250	total: 939ms	remaining: 1m 24s
11:	learn: 0.0602038	total: 999ms	remaining: 1m 22s
12:	learn: 0.0601772	total: 1.05s	remaining: 1m 19s
13:	learn: 0.0601557	total: 1.09s	remaining: 1m 16s
14:	learn: 0.0601316	total: 1.16s	remaining: 1m 16s
15:	learn: 0.0601080	total: 1.21s	remaining: 1m 14s
16:	learn: 0.0600863	total: 1.25s	remaining: 1m 12s
17:	learn: 0.0600685	total: 1.29s	remaining: 1m 10s
18:	learn: 0.0600514	total: 1.34s	remaining: 1m 9s
19:	learn: 0.0600280	tot

161:	learn: 0.0588242	total: 9.64s	remaining: 49.9s
162:	learn: 0.0588166	total: 9.74s	remaining: 50s
163:	learn: 0.0588126	total: 9.78s	remaining: 49.9s
164:	learn: 0.0588096	total: 9.83s	remaining: 49.8s
165:	learn: 0.0588054	total: 9.92s	remaining: 49.8s
166:	learn: 0.0587975	total: 10s	remaining: 49.9s
167:	learn: 0.0587928	total: 10.1s	remaining: 49.9s
168:	learn: 0.0587857	total: 10.2s	remaining: 50s
169:	learn: 0.0587806	total: 10.2s	remaining: 50s
170:	learn: 0.0587762	total: 10.3s	remaining: 49.9s
171:	learn: 0.0587749	total: 10.3s	remaining: 49.7s
172:	learn: 0.0587682	total: 10.4s	remaining: 49.6s
173:	learn: 0.0587642	total: 10.4s	remaining: 49.5s
174:	learn: 0.0587611	total: 10.5s	remaining: 49.3s
175:	learn: 0.0587576	total: 10.5s	remaining: 49.2s
176:	learn: 0.0587540	total: 10.5s	remaining: 49s
177:	learn: 0.0587517	total: 10.6s	remaining: 48.9s
178:	learn: 0.0587490	total: 10.6s	remaining: 48.8s
179:	learn: 0.0587448	total: 10.7s	remaining: 48.7s
180:	learn: 0.0587377	

320:	learn: 0.0580728	total: 20.4s	remaining: 43.1s
321:	learn: 0.0580677	total: 20.4s	remaining: 43s
322:	learn: 0.0580666	total: 20.5s	remaining: 42.9s
323:	learn: 0.0580639	total: 20.5s	remaining: 42.8s
324:	learn: 0.0580566	total: 20.6s	remaining: 42.8s
325:	learn: 0.0580539	total: 20.6s	remaining: 42.7s
326:	learn: 0.0580507	total: 20.7s	remaining: 42.6s
327:	learn: 0.0580479	total: 20.7s	remaining: 42.5s
328:	learn: 0.0580453	total: 20.8s	remaining: 42.4s
329:	learn: 0.0580436	total: 20.8s	remaining: 42.3s
330:	learn: 0.0580410	total: 20.9s	remaining: 42.2s
331:	learn: 0.0580375	total: 20.9s	remaining: 42.1s
332:	learn: 0.0580341	total: 21s	remaining: 42s
333:	learn: 0.0580309	total: 21s	remaining: 41.9s
334:	learn: 0.0580280	total: 21.1s	remaining: 41.8s
335:	learn: 0.0580211	total: 21.1s	remaining: 41.7s
336:	learn: 0.0580159	total: 21.2s	remaining: 41.6s
337:	learn: 0.0580125	total: 21.2s	remaining: 41.6s
338:	learn: 0.0580097	total: 21.3s	remaining: 41.5s
339:	learn: 0.058003

481:	learn: 0.0574361	total: 29s	remaining: 31.1s
482:	learn: 0.0574327	total: 29s	remaining: 31s
483:	learn: 0.0574283	total: 29.1s	remaining: 31s
484:	learn: 0.0574255	total: 29.1s	remaining: 30.9s
485:	learn: 0.0574223	total: 29.1s	remaining: 30.8s
486:	learn: 0.0574215	total: 29.2s	remaining: 30.8s
487:	learn: 0.0574177	total: 29.2s	remaining: 30.7s
488:	learn: 0.0574160	total: 29.3s	remaining: 30.6s
489:	learn: 0.0574145	total: 29.3s	remaining: 30.5s
490:	learn: 0.0574127	total: 29.4s	remaining: 30.4s
491:	learn: 0.0574107	total: 29.4s	remaining: 30.4s
492:	learn: 0.0574063	total: 29.5s	remaining: 30.3s
493:	learn: 0.0574006	total: 29.5s	remaining: 30.2s
494:	learn: 0.0573977	total: 29.6s	remaining: 30.2s
495:	learn: 0.0573886	total: 29.6s	remaining: 30.1s
496:	learn: 0.0573848	total: 29.7s	remaining: 30.1s
497:	learn: 0.0573805	total: 29.8s	remaining: 30s
498:	learn: 0.0573795	total: 29.8s	remaining: 29.9s
499:	learn: 0.0573757	total: 29.8s	remaining: 29.8s
500:	learn: 0.0573741	

640:	learn: 0.0568774	total: 37.7s	remaining: 21.1s
641:	learn: 0.0568749	total: 37.8s	remaining: 21.1s
642:	learn: 0.0568722	total: 37.8s	remaining: 21s
643:	learn: 0.0568692	total: 37.9s	remaining: 20.9s
644:	learn: 0.0568645	total: 37.9s	remaining: 20.9s
645:	learn: 0.0568630	total: 38s	remaining: 20.8s
646:	learn: 0.0568609	total: 38s	remaining: 20.7s
647:	learn: 0.0568594	total: 38s	remaining: 20.7s
648:	learn: 0.0568533	total: 38.1s	remaining: 20.6s
649:	learn: 0.0568481	total: 38.1s	remaining: 20.5s
650:	learn: 0.0568411	total: 38.2s	remaining: 20.5s
651:	learn: 0.0568377	total: 38.2s	remaining: 20.4s
652:	learn: 0.0568341	total: 38.3s	remaining: 20.3s
653:	learn: 0.0568314	total: 38.3s	remaining: 20.3s
654:	learn: 0.0568304	total: 38.4s	remaining: 20.2s
655:	learn: 0.0568254	total: 38.4s	remaining: 20.2s
656:	learn: 0.0568211	total: 38.5s	remaining: 20.1s
657:	learn: 0.0568150	total: 38.5s	remaining: 20s
658:	learn: 0.0568125	total: 38.6s	remaining: 20s
659:	learn: 0.0568098	to

799:	learn: 0.0563597	total: 45.8s	remaining: 11.4s
800:	learn: 0.0563555	total: 45.8s	remaining: 11.4s
801:	learn: 0.0563545	total: 45.9s	remaining: 11.3s
802:	learn: 0.0563486	total: 45.9s	remaining: 11.3s
803:	learn: 0.0563466	total: 46s	remaining: 11.2s
804:	learn: 0.0563411	total: 46s	remaining: 11.1s
805:	learn: 0.0563370	total: 46.1s	remaining: 11.1s
806:	learn: 0.0563304	total: 46.1s	remaining: 11s
807:	learn: 0.0563289	total: 46.2s	remaining: 11s
808:	learn: 0.0563276	total: 46.3s	remaining: 10.9s
809:	learn: 0.0563245	total: 46.3s	remaining: 10.9s
810:	learn: 0.0563216	total: 46.4s	remaining: 10.8s
811:	learn: 0.0563193	total: 46.4s	remaining: 10.7s
812:	learn: 0.0563163	total: 46.4s	remaining: 10.7s
813:	learn: 0.0563114	total: 46.5s	remaining: 10.6s
814:	learn: 0.0563077	total: 46.6s	remaining: 10.6s
815:	learn: 0.0563045	total: 46.6s	remaining: 10.5s
816:	learn: 0.0563015	total: 46.6s	remaining: 10.4s
817:	learn: 0.0562978	total: 46.7s	remaining: 10.4s
818:	learn: 0.056295

961:	learn: 0.0558603	total: 54s	remaining: 2.13s
962:	learn: 0.0558579	total: 54.1s	remaining: 2.08s
963:	learn: 0.0558545	total: 54.1s	remaining: 2.02s
964:	learn: 0.0558488	total: 54.2s	remaining: 1.96s
965:	learn: 0.0558437	total: 54.2s	remaining: 1.91s
966:	learn: 0.0558418	total: 54.3s	remaining: 1.85s
967:	learn: 0.0558398	total: 54.3s	remaining: 1.79s
968:	learn: 0.0558377	total: 54.4s	remaining: 1.74s
969:	learn: 0.0558340	total: 54.4s	remaining: 1.68s
970:	learn: 0.0558316	total: 54.5s	remaining: 1.63s
971:	learn: 0.0558286	total: 54.5s	remaining: 1.57s
972:	learn: 0.0558272	total: 54.6s	remaining: 1.51s
973:	learn: 0.0558248	total: 54.6s	remaining: 1.46s
974:	learn: 0.0558192	total: 54.7s	remaining: 1.4s
975:	learn: 0.0558144	total: 54.7s	remaining: 1.35s
976:	learn: 0.0558109	total: 54.8s	remaining: 1.29s
977:	learn: 0.0558082	total: 55s	remaining: 1.24s
978:	learn: 0.0558059	total: 55s	remaining: 1.18s
979:	learn: 0.0558030	total: 55.2s	remaining: 1.13s
980:	learn: 0.05580

In [19]:
for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = model_cat_y_pred
    
sample.to_csv('./data/model_cat_sample_p', index=False,float_format='%.5f')

### Apply the best parameter from RandomizeCV back to the model

iterations=5000 produce the same score as iterations=1000, which is ideal for modeling as less computing power is needed.

In [20]:
from catboost import CatBoostRegressor
model_cat_CV = CatBoostRegressor(iterations=5000,learning_rate=0.01, depth=2, loss_function='MAE', 
                          eval_metric='MAE', random_seed=2)

# fit model
model_cat_CV.fit(X_train, y_train)

# prediction
model_cat_CV_y_pred = model_cat_CV.predict(X_test)

0:	learn: 0.0608724	total: 24.6ms	remaining: 2m 2s
1:	learn: 0.0607971	total: 47.8ms	remaining: 1m 59s
2:	learn: 0.0607337	total: 69.8ms	remaining: 1m 56s
3:	learn: 0.0606857	total: 85ms	remaining: 1m 46s
4:	learn: 0.0606445	total: 102ms	remaining: 1m 41s
5:	learn: 0.0606079	total: 117ms	remaining: 1m 37s
6:	learn: 0.0605754	total: 132ms	remaining: 1m 34s
7:	learn: 0.0605500	total: 147ms	remaining: 1m 31s
8:	learn: 0.0605270	total: 163ms	remaining: 1m 30s
9:	learn: 0.0605083	total: 178ms	remaining: 1m 28s
10:	learn: 0.0604892	total: 194ms	remaining: 1m 28s
11:	learn: 0.0604722	total: 210ms	remaining: 1m 27s
12:	learn: 0.0604600	total: 232ms	remaining: 1m 28s
13:	learn: 0.0604455	total: 253ms	remaining: 1m 30s
14:	learn: 0.0604331	total: 282ms	remaining: 1m 33s
15:	learn: 0.0604240	total: 300ms	remaining: 1m 33s
16:	learn: 0.0604151	total: 314ms	remaining: 1m 32s
17:	learn: 0.0604058	total: 328ms	remaining: 1m 30s
18:	learn: 0.0603973	total: 344ms	remaining: 1m 30s
19:	learn: 0.0603914	

158:	learn: 0.0600968	total: 2.73s	remaining: 1m 23s
159:	learn: 0.0600963	total: 2.76s	remaining: 1m 23s
160:	learn: 0.0600951	total: 2.78s	remaining: 1m 23s
161:	learn: 0.0600944	total: 2.8s	remaining: 1m 23s
162:	learn: 0.0600941	total: 2.82s	remaining: 1m 23s
163:	learn: 0.0600935	total: 2.84s	remaining: 1m 23s
164:	learn: 0.0600928	total: 2.85s	remaining: 1m 23s
165:	learn: 0.0600918	total: 2.87s	remaining: 1m 23s
166:	learn: 0.0600909	total: 2.88s	remaining: 1m 23s
167:	learn: 0.0600902	total: 2.9s	remaining: 1m 23s
168:	learn: 0.0600895	total: 2.91s	remaining: 1m 23s
169:	learn: 0.0600881	total: 2.93s	remaining: 1m 23s
170:	learn: 0.0600877	total: 2.94s	remaining: 1m 23s
171:	learn: 0.0600870	total: 2.97s	remaining: 1m 23s
172:	learn: 0.0600857	total: 2.99s	remaining: 1m 23s
173:	learn: 0.0600851	total: 3.01s	remaining: 1m 23s
174:	learn: 0.0600841	total: 3.02s	remaining: 1m 23s
175:	learn: 0.0600833	total: 3.04s	remaining: 1m 23s
176:	learn: 0.0600824	total: 3.05s	remaining: 1m

317:	learn: 0.0599886	total: 5.47s	remaining: 1m 20s
318:	learn: 0.0599881	total: 5.5s	remaining: 1m 20s
319:	learn: 0.0599876	total: 5.51s	remaining: 1m 20s
320:	learn: 0.0599873	total: 5.52s	remaining: 1m 20s
321:	learn: 0.0599867	total: 5.54s	remaining: 1m 20s
322:	learn: 0.0599863	total: 5.56s	remaining: 1m 20s
323:	learn: 0.0599860	total: 5.57s	remaining: 1m 20s
324:	learn: 0.0599855	total: 5.59s	remaining: 1m 20s
325:	learn: 0.0599846	total: 5.6s	remaining: 1m 20s
326:	learn: 0.0599837	total: 5.62s	remaining: 1m 20s
327:	learn: 0.0599833	total: 5.63s	remaining: 1m 20s
328:	learn: 0.0599829	total: 5.65s	remaining: 1m 20s
329:	learn: 0.0599827	total: 5.66s	remaining: 1m 20s
330:	learn: 0.0599824	total: 5.68s	remaining: 1m 20s
331:	learn: 0.0599810	total: 5.7s	remaining: 1m 20s
332:	learn: 0.0599803	total: 5.71s	remaining: 1m 20s
333:	learn: 0.0599801	total: 5.73s	remaining: 1m 20s
334:	learn: 0.0599800	total: 5.75s	remaining: 1m 20s
335:	learn: 0.0599786	total: 5.76s	remaining: 1m 

477:	learn: 0.0599244	total: 8.16s	remaining: 1m 17s
478:	learn: 0.0599242	total: 8.17s	remaining: 1m 17s
479:	learn: 0.0599236	total: 8.2s	remaining: 1m 17s
480:	learn: 0.0599233	total: 8.21s	remaining: 1m 17s
481:	learn: 0.0599227	total: 8.23s	remaining: 1m 17s
482:	learn: 0.0599218	total: 8.24s	remaining: 1m 17s
483:	learn: 0.0599210	total: 8.26s	remaining: 1m 17s
484:	learn: 0.0599205	total: 8.28s	remaining: 1m 17s
485:	learn: 0.0599198	total: 8.29s	remaining: 1m 17s
486:	learn: 0.0599197	total: 8.31s	remaining: 1m 16s
487:	learn: 0.0599189	total: 8.32s	remaining: 1m 16s
488:	learn: 0.0599187	total: 8.33s	remaining: 1m 16s
489:	learn: 0.0599187	total: 8.35s	remaining: 1m 16s
490:	learn: 0.0599176	total: 8.36s	remaining: 1m 16s
491:	learn: 0.0599175	total: 8.38s	remaining: 1m 16s
492:	learn: 0.0599168	total: 8.41s	remaining: 1m 16s
493:	learn: 0.0599165	total: 8.43s	remaining: 1m 16s
494:	learn: 0.0599152	total: 8.45s	remaining: 1m 16s
495:	learn: 0.0599150	total: 8.47s	remaining: 1

632:	learn: 0.0598695	total: 10.6s	remaining: 1m 13s
633:	learn: 0.0598694	total: 10.6s	remaining: 1m 13s
634:	learn: 0.0598691	total: 10.6s	remaining: 1m 13s
635:	learn: 0.0598684	total: 10.6s	remaining: 1m 13s
636:	learn: 0.0598682	total: 10.7s	remaining: 1m 13s
637:	learn: 0.0598678	total: 10.7s	remaining: 1m 12s
638:	learn: 0.0598672	total: 10.7s	remaining: 1m 12s
639:	learn: 0.0598670	total: 10.7s	remaining: 1m 12s
640:	learn: 0.0598667	total: 10.7s	remaining: 1m 12s
641:	learn: 0.0598667	total: 10.7s	remaining: 1m 12s
642:	learn: 0.0598660	total: 10.7s	remaining: 1m 12s
643:	learn: 0.0598660	total: 10.8s	remaining: 1m 12s
644:	learn: 0.0598655	total: 10.8s	remaining: 1m 12s
645:	learn: 0.0598653	total: 10.8s	remaining: 1m 12s
646:	learn: 0.0598651	total: 10.8s	remaining: 1m 12s
647:	learn: 0.0598645	total: 10.8s	remaining: 1m 12s
648:	learn: 0.0598644	total: 10.8s	remaining: 1m 12s
649:	learn: 0.0598642	total: 10.9s	remaining: 1m 12s
650:	learn: 0.0598639	total: 10.9s	remaining: 

...

4329:	learn: 0.0592389	total: 1m 8s	remaining: 10.5s
4330:	learn: 0.0592387	total: 1m 8s	remaining: 10.5s
4331:	learn: 0.0592386	total: 1m 8s	remaining: 10.5s
4332:	learn: 0.0592385	total: 1m 8s	remaining: 10.5s
4333:	learn: 0.0592383	total: 1m 8s	remaining: 10.5s
4334:	learn: 0.0592382	total: 1m 8s	remaining: 10.4s
4335:	learn: 0.0592381	total: 1m 8s	remaining: 10.4s
4336:	learn: 0.0592379	total: 1m 8s	remaining: 10.4s
4337:	learn: 0.0592378	total: 1m 8s	remaining: 10.4s
4338:	learn: 0.0592377	total: 1m 8s	remaining: 10.4s
4339:	learn: 0.0592376	total: 1m 8s	remaining: 10.4s
4340:	learn: 0.0592376	total: 1m 8s	remaining: 10.4s
4341:	learn: 0.0592375	total: 1m 8s	remaining: 10.3s
4342:	learn: 0.0592375	total: 1m 8s	remaining: 10.3s
4343:	learn: 0.0592374	total: 1m 8s	remaining: 10.3s
4344:	learn: 0.0592373	total: 1m 8s	remaining: 10.3s
4345:	learn: 0.0592371	total: 1m 8s	remaining: 10.3s
4346:	learn: 0.0592369	total: 1m 8s	remaining: 10.3s
4347:	learn: 0.0592368	total: 1m 8s	remaining:

4487:	learn: 0.0592231	total: 1m 10s	remaining: 8.04s
4488:	learn: 0.0592230	total: 1m 10s	remaining: 8.03s
4489:	learn: 0.0592228	total: 1m 10s	remaining: 8.01s
4490:	learn: 0.0592226	total: 1m 10s	remaining: 7.99s
4491:	learn: 0.0592223	total: 1m 10s	remaining: 7.98s
4492:	learn: 0.0592222	total: 1m 10s	remaining: 7.96s
4493:	learn: 0.0592220	total: 1m 10s	remaining: 7.95s
4494:	learn: 0.0592219	total: 1m 10s	remaining: 7.93s
4495:	learn: 0.0592219	total: 1m 10s	remaining: 7.91s
4496:	learn: 0.0592218	total: 1m 10s	remaining: 7.9s
4497:	learn: 0.0592216	total: 1m 10s	remaining: 7.88s
4498:	learn: 0.0592215	total: 1m 10s	remaining: 7.87s
4499:	learn: 0.0592214	total: 1m 10s	remaining: 7.85s
4500:	learn: 0.0592213	total: 1m 10s	remaining: 7.84s
4501:	learn: 0.0592212	total: 1m 10s	remaining: 7.82s
4502:	learn: 0.0592210	total: 1m 10s	remaining: 7.8s
4503:	learn: 0.0592209	total: 1m 10s	remaining: 7.79s
4504:	learn: 0.0592209	total: 1m 10s	remaining: 7.77s
4505:	learn: 0.0592205	total: 

4645:	learn: 0.0592062	total: 1m 12s	remaining: 5.56s
4646:	learn: 0.0592061	total: 1m 12s	remaining: 5.54s
4647:	learn: 0.0592061	total: 1m 12s	remaining: 5.53s
4648:	learn: 0.0592059	total: 1m 12s	remaining: 5.51s
4649:	learn: 0.0592057	total: 1m 13s	remaining: 5.5s
4650:	learn: 0.0592057	total: 1m 13s	remaining: 5.48s
4651:	learn: 0.0592056	total: 1m 13s	remaining: 5.46s
4652:	learn: 0.0592055	total: 1m 13s	remaining: 5.45s
4653:	learn: 0.0592055	total: 1m 13s	remaining: 5.43s
4654:	learn: 0.0592054	total: 1m 13s	remaining: 5.42s
4655:	learn: 0.0592053	total: 1m 13s	remaining: 5.4s
4656:	learn: 0.0592052	total: 1m 13s	remaining: 5.38s
4657:	learn: 0.0592051	total: 1m 13s	remaining: 5.37s
4658:	learn: 0.0592050	total: 1m 13s	remaining: 5.35s
4659:	learn: 0.0592050	total: 1m 13s	remaining: 5.34s
4660:	learn: 0.0592049	total: 1m 13s	remaining: 5.32s
4661:	learn: 0.0592047	total: 1m 13s	remaining: 5.31s
4662:	learn: 0.0592046	total: 1m 13s	remaining: 5.29s
4663:	learn: 0.0592046	total: 

4805:	learn: 0.0591903	total: 1m 15s	remaining: 3.04s
4806:	learn: 0.0591903	total: 1m 15s	remaining: 3.03s
4807:	learn: 0.0591903	total: 1m 15s	remaining: 3.01s
4808:	learn: 0.0591901	total: 1m 15s	remaining: 3s
4809:	learn: 0.0591901	total: 1m 15s	remaining: 2.98s
4810:	learn: 0.0591899	total: 1m 15s	remaining: 2.96s
4811:	learn: 0.0591898	total: 1m 15s	remaining: 2.95s
4812:	learn: 0.0591898	total: 1m 15s	remaining: 2.93s
4813:	learn: 0.0591897	total: 1m 15s	remaining: 2.92s
4814:	learn: 0.0591894	total: 1m 15s	remaining: 2.9s
4815:	learn: 0.0591894	total: 1m 15s	remaining: 2.89s
4816:	learn: 0.0591894	total: 1m 15s	remaining: 2.87s
4817:	learn: 0.0591892	total: 1m 15s	remaining: 2.85s
4818:	learn: 0.0591892	total: 1m 15s	remaining: 2.84s
4819:	learn: 0.0591892	total: 1m 15s	remaining: 2.82s
4820:	learn: 0.0591890	total: 1m 15s	remaining: 2.81s
4821:	learn: 0.0591889	total: 1m 15s	remaining: 2.79s
4822:	learn: 0.0591888	total: 1m 15s	remaining: 2.78s
4823:	learn: 0.0591888	total: 1m

4965:	learn: 0.0591753	total: 1m 17s	remaining: 533ms
4966:	learn: 0.0591752	total: 1m 17s	remaining: 517ms
4967:	learn: 0.0591752	total: 1m 17s	remaining: 502ms
4968:	learn: 0.0591751	total: 1m 17s	remaining: 486ms
4969:	learn: 0.0591750	total: 1m 17s	remaining: 470ms
4970:	learn: 0.0591749	total: 1m 17s	remaining: 455ms
4971:	learn: 0.0591749	total: 1m 17s	remaining: 439ms
4972:	learn: 0.0591748	total: 1m 17s	remaining: 423ms
4973:	learn: 0.0591747	total: 1m 17s	remaining: 408ms
4974:	learn: 0.0591747	total: 1m 17s	remaining: 392ms
4975:	learn: 0.0591746	total: 1m 18s	remaining: 376ms
4976:	learn: 0.0591745	total: 1m 18s	remaining: 361ms
4977:	learn: 0.0591745	total: 1m 18s	remaining: 345ms
4978:	learn: 0.0591744	total: 1m 18s	remaining: 329ms
4979:	learn: 0.0591743	total: 1m 18s	remaining: 314ms
4980:	learn: 0.0591742	total: 1m 18s	remaining: 298ms
4981:	learn: 0.0591741	total: 1m 18s	remaining: 282ms
4982:	learn: 0.0591740	total: 1m 18s	remaining: 267ms
4983:	learn: 0.0591740	total

In [21]:
sample = pd.read_pickle('./data/sample_p')

for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = model_cat_CV_y_pred
    
sample.to_csv('./data/model_cat_RandomizeCV_sample_p', index=False,float_format='%.5f')

In [22]:
cat_feature_importances = model_cat_CV.feature_importances_

cat_feature_importances = pd.DataFrame(cat_feature_importances,
                                   index = test_columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
cat_feature_importances

,importance
calculatedfinishedsquarefeet,12.729238
N_property_tax_value_ratio,6.583681
taxamount,6.056299
lotsizesquarefeet,5.050657
N_location,4.894671
regionidzip,4.624552
latitude,4.604952
N_structure_value_ratio,3.598795
yearbuilt,3.546567
regionidcity,3.290339


# LightGBM

In [17]:
import lightgbm as lgb

In [18]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
#lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    #'objective': 'regression',
    'metric': {'mae'},
    #'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

# train model
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20)

# predict
gbm_y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [19]:
sample = pd.read_pickle('./data/sample_p')

for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = gbm_y_pred
    
sample.to_csv('./data/gbm_y_pred_sample_p', index=False,float_format='%.5f')

In [ ]:
---

# Ensemble model

## Tune Hyperparameters
Each model improves significantly after hyperparameter tuning, compared with the training by default settings. The performance for these three models can be improved further, if we put more possible parameter settings, which will also increase the running time.

In [84]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from catboost import CatBoostRegressor

# models
model_cat = CatBoostRegressor
model_gbm = lgb.LGBMRegressor

# pairs list
model_cat_params = {
    'iterations':[5000],
    'learning_rate':np.arange(0.01,0.02,0.01),
    'depth':np.arange(3,5,1)
}


model_gbm_params = {'learning_rate':np.arange(0.05,0.06,0.01),
'depth':np.arange(3,5,1)}

clf_params = [(model_cat,model_cat_params),
              (model_gbm,model_gbm_params)
              ]

In [85]:
# hyperparameter tuning
import time

tune_results = {}

for clf, params in clf_params:
    
    start = time.time()
    tune = GridSearchCV(clf(), params,cv=3,verbose=True,n_jobs=-1)
    tune.fit(X_train, y_train)
    best_estimator = tune.best_estimator_
    best_params = tune.best_params_
    best_score = tune.best_score_
    tune_results[clf] = (best_estimator, best_params, best_score, (time.time()-start)/60)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  7.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  7.4min finished


0:	learn: 0.1107519	total: 20.9ms	remaining: 1m 44s
1:	learn: 0.1107421	total: 39ms	remaining: 1m 37s
2:	learn: 0.1107334	total: 56.6ms	remaining: 1m 34s
3:	learn: 0.1107228	total: 82.5ms	remaining: 1m 42s
4:	learn: 0.1107141	total: 110ms	remaining: 1m 50s
5:	learn: 0.1107055	total: 139ms	remaining: 1m 55s
6:	learn: 0.1106961	total: 168ms	remaining: 1m 59s
7:	learn: 0.1106880	total: 199ms	remaining: 2m 3s
8:	learn: 0.1106808	total: 229ms	remaining: 2m 6s
9:	learn: 0.1106727	total: 262ms	remaining: 2m 10s
10:	learn: 0.1106650	total: 282ms	remaining: 2m 8s
11:	learn: 0.1106570	total: 299ms	remaining: 2m 4s
12:	learn: 0.1106498	total: 314ms	remaining: 2m
13:	learn: 0.1106419	total: 330ms	remaining: 1m 57s
14:	learn: 0.1106348	total: 347ms	remaining: 1m 55s
15:	learn: 0.1106267	total: 363ms	remaining: 1m 52s
16:	learn: 0.1106191	total: 379ms	remaining: 1m 51s
17:	learn: 0.1106119	total: 395ms	remaining: 1m 49s
18:	learn: 0.1106046	total: 412ms	remaining: 1m 47s
19:	learn: 0.1105972	total: 

166:	learn: 0.1101204	total: 3.14s	remaining: 1m 30s
167:	learn: 0.1101191	total: 3.17s	remaining: 1m 31s
168:	learn: 0.1101175	total: 3.19s	remaining: 1m 31s
169:	learn: 0.1101165	total: 3.2s	remaining: 1m 30s
170:	learn: 0.1101154	total: 3.22s	remaining: 1m 30s
171:	learn: 0.1101139	total: 3.24s	remaining: 1m 30s
172:	learn: 0.1101128	total: 3.25s	remaining: 1m 30s
173:	learn: 0.1101112	total: 3.27s	remaining: 1m 30s
174:	learn: 0.1101093	total: 3.29s	remaining: 1m 30s
175:	learn: 0.1101078	total: 3.31s	remaining: 1m 30s
176:	learn: 0.1101061	total: 3.33s	remaining: 1m 30s
177:	learn: 0.1101045	total: 3.34s	remaining: 1m 30s
178:	learn: 0.1101032	total: 3.36s	remaining: 1m 30s
179:	learn: 0.1101017	total: 3.38s	remaining: 1m 30s
180:	learn: 0.1100999	total: 3.4s	remaining: 1m 30s
181:	learn: 0.1100988	total: 3.42s	remaining: 1m 30s
182:	learn: 0.1100970	total: 3.44s	remaining: 1m 30s
183:	learn: 0.1100956	total: 3.46s	remaining: 1m 30s
184:	learn: 0.1100942	total: 3.48s	remaining: 1m

329:	learn: 0.1099360	total: 6.05s	remaining: 1m 25s
330:	learn: 0.1099354	total: 6.07s	remaining: 1m 25s
331:	learn: 0.1099348	total: 6.08s	remaining: 1m 25s
332:	learn: 0.1099339	total: 6.1s	remaining: 1m 25s
333:	learn: 0.1099330	total: 6.12s	remaining: 1m 25s
334:	learn: 0.1099321	total: 6.13s	remaining: 1m 25s
335:	learn: 0.1099313	total: 6.15s	remaining: 1m 25s
336:	learn: 0.1099305	total: 6.17s	remaining: 1m 25s
337:	learn: 0.1099297	total: 6.18s	remaining: 1m 25s
338:	learn: 0.1099291	total: 6.2s	remaining: 1m 25s
339:	learn: 0.1099277	total: 6.22s	remaining: 1m 25s
340:	learn: 0.1099264	total: 6.24s	remaining: 1m 25s
341:	learn: 0.1099253	total: 6.26s	remaining: 1m 25s
342:	learn: 0.1099245	total: 6.28s	remaining: 1m 25s
343:	learn: 0.1099239	total: 6.3s	remaining: 1m 25s
344:	learn: 0.1099224	total: 6.31s	remaining: 1m 25s
345:	learn: 0.1099216	total: 6.33s	remaining: 1m 25s
346:	learn: 0.1099205	total: 6.35s	remaining: 1m 25s
347:	learn: 0.1099198	total: 6.37s	remaining: 1m 

...

4383:	learn: 0.1086353	total: 1m 13s	remaining: 10.3s
4384:	learn: 0.1086352	total: 1m 13s	remaining: 10.3s
4385:	learn: 0.1086351	total: 1m 13s	remaining: 10.3s
4386:	learn: 0.1086345	total: 1m 13s	remaining: 10.3s
4387:	learn: 0.1086340	total: 1m 13s	remaining: 10.2s
4388:	learn: 0.1086338	total: 1m 13s	remaining: 10.2s
4389:	learn: 0.1086337	total: 1m 13s	remaining: 10.2s
4390:	learn: 0.1086334	total: 1m 13s	remaining: 10.2s
4391:	learn: 0.1086331	total: 1m 13s	remaining: 10.2s
4392:	learn: 0.1086325	total: 1m 13s	remaining: 10.2s
4393:	learn: 0.1086322	total: 1m 13s	remaining: 10.1s
4394:	learn: 0.1086322	total: 1m 13s	remaining: 10.1s
4395:	learn: 0.1086320	total: 1m 13s	remaining: 10.1s
4396:	learn: 0.1086318	total: 1m 13s	remaining: 10.1s
4397:	learn: 0.1086315	total: 1m 13s	remaining: 10.1s
4398:	learn: 0.1086312	total: 1m 13s	remaining: 10.1s
4399:	learn: 0.1086310	total: 1m 13s	remaining: 10s
4400:	learn: 0.1086308	total: 1m 13s	remaining: 10s
4401:	learn: 0.1086306	total: 1m

4545:	learn: 0.1086006	total: 1m 15s	remaining: 7.59s
4546:	learn: 0.1086004	total: 1m 16s	remaining: 7.57s
4547:	learn: 0.1086004	total: 1m 16s	remaining: 7.56s
4548:	learn: 0.1086003	total: 1m 16s	remaining: 7.54s
4549:	learn: 0.1086002	total: 1m 16s	remaining: 7.52s
4550:	learn: 0.1086000	total: 1m 16s	remaining: 7.5s
4551:	learn: 0.1085998	total: 1m 16s	remaining: 7.49s
4552:	learn: 0.1085997	total: 1m 16s	remaining: 7.47s
4553:	learn: 0.1085995	total: 1m 16s	remaining: 7.45s
4554:	learn: 0.1085995	total: 1m 16s	remaining: 7.44s
4555:	learn: 0.1085991	total: 1m 16s	remaining: 7.42s
4556:	learn: 0.1085989	total: 1m 16s	remaining: 7.4s
4557:	learn: 0.1085988	total: 1m 16s	remaining: 7.39s
4558:	learn: 0.1085987	total: 1m 16s	remaining: 7.37s
4559:	learn: 0.1085985	total: 1m 16s	remaining: 7.35s
4560:	learn: 0.1085983	total: 1m 16s	remaining: 7.34s
4561:	learn: 0.1085981	total: 1m 16s	remaining: 7.32s
4562:	learn: 0.1085977	total: 1m 16s	remaining: 7.3s
4563:	learn: 0.1085976	total: 1

4708:	learn: 0.1085679	total: 1m 18s	remaining: 4.86s
4709:	learn: 0.1085676	total: 1m 18s	remaining: 4.84s
4710:	learn: 0.1085673	total: 1m 18s	remaining: 4.83s
4711:	learn: 0.1085671	total: 1m 18s	remaining: 4.81s
4712:	learn: 0.1085665	total: 1m 18s	remaining: 4.79s
4713:	learn: 0.1085663	total: 1m 18s	remaining: 4.78s
4714:	learn: 0.1085662	total: 1m 18s	remaining: 4.76s
4715:	learn: 0.1085659	total: 1m 18s	remaining: 4.74s
4716:	learn: 0.1085657	total: 1m 18s	remaining: 4.73s
4717:	learn: 0.1085655	total: 1m 18s	remaining: 4.71s
4718:	learn: 0.1085653	total: 1m 18s	remaining: 4.69s
4719:	learn: 0.1085652	total: 1m 18s	remaining: 4.68s
4720:	learn: 0.1085650	total: 1m 18s	remaining: 4.66s
4721:	learn: 0.1085649	total: 1m 18s	remaining: 4.64s
4722:	learn: 0.1085648	total: 1m 18s	remaining: 4.63s
4723:	learn: 0.1085647	total: 1m 18s	remaining: 4.61s
4724:	learn: 0.1085643	total: 1m 18s	remaining: 4.59s
4725:	learn: 0.1085642	total: 1m 18s	remaining: 4.58s
4726:	learn: 0.1085641	total

4868:	learn: 0.1085327	total: 1m 21s	remaining: 2.19s
4869:	learn: 0.1085324	total: 1m 21s	remaining: 2.17s
4870:	learn: 0.1085322	total: 1m 21s	remaining: 2.15s
4871:	learn: 0.1085319	total: 1m 21s	remaining: 2.14s
4872:	learn: 0.1085316	total: 1m 21s	remaining: 2.12s
4873:	learn: 0.1085315	total: 1m 21s	remaining: 2.1s
4874:	learn: 0.1085311	total: 1m 21s	remaining: 2.09s
4875:	learn: 0.1085310	total: 1m 21s	remaining: 2.07s
4876:	learn: 0.1085308	total: 1m 21s	remaining: 2.05s
4877:	learn: 0.1085307	total: 1m 21s	remaining: 2.04s
4878:	learn: 0.1085306	total: 1m 21s	remaining: 2.02s
4879:	learn: 0.1085304	total: 1m 21s	remaining: 2s
4880:	learn: 0.1085303	total: 1m 21s	remaining: 1.99s
4881:	learn: 0.1085299	total: 1m 21s	remaining: 1.97s
4882:	learn: 0.1085297	total: 1m 21s	remaining: 1.95s
4883:	learn: 0.1085295	total: 1m 21s	remaining: 1.94s
4884:	learn: 0.1085293	total: 1m 21s	remaining: 1.92s
4885:	learn: 0.1085292	total: 1m 21s	remaining: 1.9s
4886:	learn: 0.1085289	total: 1m 

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    8.2s finished


## Stacking

In [86]:
from sklearn.model_selection import KFold
import scipy

def get_oof(clf, x_train, y_train, x_test):
    
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    kf = KFold(n_splits= 3, random_state=42)
    
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((3, ntest))
    i = 0
    for train_index, test_index in kf.split(x_train):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)
        
        i += 1
    
    oof_test[:] = scipy.stats.mode(oof_test_skf, axis=0)[0]
    
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [87]:
stack_train = []
stack_test = []

for clf, _ in clf_params:
    tmp = clf(**tune_results[clf][1])
    oof_train, oof_test = get_oof(tmp, X_train, y_train, X_test)
    stack_train.append(oof_train)
    stack_test.append(oof_test)
    
X_train_stacked = np.concatenate(stack_train, axis=1)
X_test_stacked = np.concatenate(stack_test, axis=1)

# meta classifier
metas = {}
for clf, _ in clf_params:
    metas[clf] = clf().fit(X_train_stacked, y_train)

0:	learn: 0.1098339	total: 19.9ms	remaining: 1m 39s
1:	learn: 0.1098264	total: 33.9ms	remaining: 1m 24s
2:	learn: 0.1098183	total: 52.6ms	remaining: 1m 27s
3:	learn: 0.1098112	total: 63.1ms	remaining: 1m 18s
4:	learn: 0.1098011	total: 74.4ms	remaining: 1m 14s
5:	learn: 0.1097930	total: 85.1ms	remaining: 1m 10s
6:	learn: 0.1097846	total: 97.2ms	remaining: 1m 9s
7:	learn: 0.1097768	total: 108ms	remaining: 1m 7s
8:	learn: 0.1097693	total: 120ms	remaining: 1m 6s
9:	learn: 0.1097620	total: 131ms	remaining: 1m 5s
10:	learn: 0.1097534	total: 142ms	remaining: 1m 4s
11:	learn: 0.1097460	total: 153ms	remaining: 1m 3s
12:	learn: 0.1097382	total: 166ms	remaining: 1m 3s
13:	learn: 0.1097328	total: 177ms	remaining: 1m 2s
14:	learn: 0.1097255	total: 188ms	remaining: 1m 2s
15:	learn: 0.1097169	total: 199ms	remaining: 1m 2s
16:	learn: 0.1097090	total: 211ms	remaining: 1m 1s
17:	learn: 0.1097019	total: 225ms	remaining: 1m 2s
18:	learn: 0.1096947	total: 251ms	remaining: 1m 5s
19:	learn: 0.1096873	total: 

164:	learn: 0.1092380	total: 2.49s	remaining: 1m 12s
165:	learn: 0.1092368	total: 2.5s	remaining: 1m 12s
166:	learn: 0.1092361	total: 2.52s	remaining: 1m 12s
167:	learn: 0.1092344	total: 2.53s	remaining: 1m 12s
168:	learn: 0.1092336	total: 2.55s	remaining: 1m 12s
169:	learn: 0.1092326	total: 2.56s	remaining: 1m 12s
170:	learn: 0.1092306	total: 2.57s	remaining: 1m 12s
171:	learn: 0.1092292	total: 2.58s	remaining: 1m 12s
172:	learn: 0.1092276	total: 2.6s	remaining: 1m 12s
173:	learn: 0.1092259	total: 2.61s	remaining: 1m 12s
174:	learn: 0.1092240	total: 2.62s	remaining: 1m 12s
175:	learn: 0.1092224	total: 2.63s	remaining: 1m 12s
176:	learn: 0.1092202	total: 2.64s	remaining: 1m 11s
177:	learn: 0.1092195	total: 2.65s	remaining: 1m 11s
178:	learn: 0.1092181	total: 2.67s	remaining: 1m 11s
179:	learn: 0.1092162	total: 2.68s	remaining: 1m 11s
180:	learn: 0.1092144	total: 2.7s	remaining: 1m 11s
181:	learn: 0.1092129	total: 2.72s	remaining: 1m 12s
182:	learn: 0.1092113	total: 2.74s	remaining: 1m 

332:	learn: 0.1090402	total: 4.77s	remaining: 1m 6s
333:	learn: 0.1090388	total: 4.8s	remaining: 1m 7s
334:	learn: 0.1090376	total: 4.82s	remaining: 1m 7s
335:	learn: 0.1090367	total: 4.84s	remaining: 1m 7s
336:	learn: 0.1090366	total: 4.86s	remaining: 1m 7s
337:	learn: 0.1090351	total: 4.88s	remaining: 1m 7s
338:	learn: 0.1090343	total: 4.91s	remaining: 1m 7s
339:	learn: 0.1090336	total: 4.93s	remaining: 1m 7s
340:	learn: 0.1090330	total: 4.94s	remaining: 1m 7s
341:	learn: 0.1090326	total: 4.96s	remaining: 1m 7s
342:	learn: 0.1090314	total: 4.97s	remaining: 1m 7s
343:	learn: 0.1090300	total: 4.99s	remaining: 1m 7s
344:	learn: 0.1090291	total: 5.01s	remaining: 1m 7s
345:	learn: 0.1090276	total: 5.02s	remaining: 1m 7s
346:	learn: 0.1090270	total: 5.04s	remaining: 1m 7s
347:	learn: 0.1090260	total: 5.05s	remaining: 1m 7s
348:	learn: 0.1090251	total: 5.06s	remaining: 1m 7s
349:	learn: 0.1090246	total: 5.07s	remaining: 1m 7s
350:	learn: 0.1090235	total: 5.09s	remaining: 1m 7s
351:	learn: 0

501:	learn: 0.1089160	total: 7.19s	remaining: 1m 4s
502:	learn: 0.1089157	total: 7.21s	remaining: 1m 4s
503:	learn: 0.1089144	total: 7.22s	remaining: 1m 4s
504:	learn: 0.1089132	total: 7.24s	remaining: 1m 4s
505:	learn: 0.1089127	total: 7.25s	remaining: 1m 4s
506:	learn: 0.1089115	total: 7.26s	remaining: 1m 4s
507:	learn: 0.1089108	total: 7.27s	remaining: 1m 4s
508:	learn: 0.1089100	total: 7.28s	remaining: 1m 4s
509:	learn: 0.1089090	total: 7.29s	remaining: 1m 4s
510:	learn: 0.1089087	total: 7.3s	remaining: 1m 4s
511:	learn: 0.1089079	total: 7.31s	remaining: 1m 4s
512:	learn: 0.1089074	total: 7.33s	remaining: 1m 4s
513:	learn: 0.1089061	total: 7.34s	remaining: 1m 4s
514:	learn: 0.1089056	total: 7.35s	remaining: 1m 3s
515:	learn: 0.1089055	total: 7.36s	remaining: 1m 3s
516:	learn: 0.1089047	total: 7.37s	remaining: 1m 3s
517:	learn: 0.1089041	total: 7.38s	remaining: 1m 3s
518:	learn: 0.1089036	total: 7.39s	remaining: 1m 3s
519:	learn: 0.1089025	total: 7.41s	remaining: 1m 3s
520:	learn: 0

664:	learn: 0.1088110	total: 9.24s	remaining: 1m
665:	learn: 0.1088105	total: 9.26s	remaining: 1m
666:	learn: 0.1088098	total: 9.28s	remaining: 1m
667:	learn: 0.1088089	total: 9.29s	remaining: 1m
668:	learn: 0.1088087	total: 9.3s	remaining: 1m
669:	learn: 0.1088078	total: 9.31s	remaining: 1m
670:	learn: 0.1088076	total: 9.32s	remaining: 1m
671:	learn: 0.1088071	total: 9.33s	remaining: 1m
672:	learn: 0.1088067	total: 9.34s	remaining: 1m
673:	learn: 0.1088061	total: 9.35s	remaining: 1m
674:	learn: 0.1088059	total: 9.36s	remaining: 60s
675:	learn: 0.1088051	total: 9.38s	remaining: 60s
676:	learn: 0.1088046	total: 9.39s	remaining: 59.9s
677:	learn: 0.1088042	total: 9.4s	remaining: 59.9s
678:	learn: 0.1088034	total: 9.41s	remaining: 59.9s
679:	learn: 0.1088027	total: 9.42s	remaining: 59.9s
680:	learn: 0.1088012	total: 9.43s	remaining: 59.8s
681:	learn: 0.1088007	total: 9.45s	remaining: 59.8s
682:	learn: 0.1088006	total: 9.46s	remaining: 59.8s
683:	learn: 0.1087997	total: 9.47s	remaining: 59

...

274:	learn: 0.1095035	total: 4.24s	remaining: 11.2s
275:	learn: 0.1095028	total: 4.25s	remaining: 11.2s
276:	learn: 0.1095024	total: 4.27s	remaining: 11.2s
277:	learn: 0.1095016	total: 4.29s	remaining: 11.1s
278:	learn: 0.1095012	total: 4.3s	remaining: 11.1s
279:	learn: 0.1095007	total: 4.31s	remaining: 11.1s
280:	learn: 0.1095004	total: 4.33s	remaining: 11.1s
281:	learn: 0.1094999	total: 4.34s	remaining: 11.1s
282:	learn: 0.1094996	total: 4.36s	remaining: 11s
283:	learn: 0.1094995	total: 4.37s	remaining: 11s
284:	learn: 0.1094986	total: 4.38s	remaining: 11s
285:	learn: 0.1094983	total: 4.4s	remaining: 11s
286:	learn: 0.1094980	total: 4.42s	remaining: 11s
287:	learn: 0.1094977	total: 4.43s	remaining: 10.9s
288:	learn: 0.1094974	total: 4.45s	remaining: 10.9s
289:	learn: 0.1094970	total: 4.47s	remaining: 10.9s
290:	learn: 0.1094967	total: 4.48s	remaining: 10.9s
291:	learn: 0.1094965	total: 4.49s	remaining: 10.9s
292:	learn: 0.1094962	total: 4.5s	remaining: 10.9s
293:	learn: 0.1094955	tot

447:	learn: 0.1094217	total: 6.82s	remaining: 8.4s
448:	learn: 0.1094214	total: 6.83s	remaining: 8.38s
449:	learn: 0.1094202	total: 6.84s	remaining: 8.36s
450:	learn: 0.1094197	total: 6.86s	remaining: 8.35s
451:	learn: 0.1094192	total: 6.87s	remaining: 8.33s
452:	learn: 0.1094188	total: 6.88s	remaining: 8.31s
453:	learn: 0.1094185	total: 6.9s	remaining: 8.29s
454:	learn: 0.1094176	total: 6.92s	remaining: 8.28s
455:	learn: 0.1094173	total: 6.94s	remaining: 8.27s
456:	learn: 0.1094168	total: 6.95s	remaining: 8.26s
457:	learn: 0.1094163	total: 6.97s	remaining: 8.25s
458:	learn: 0.1094160	total: 6.98s	remaining: 8.23s
459:	learn: 0.1094153	total: 6.99s	remaining: 8.21s
460:	learn: 0.1094149	total: 7.01s	remaining: 8.19s
461:	learn: 0.1094145	total: 7.02s	remaining: 8.17s
462:	learn: 0.1094140	total: 7.04s	remaining: 8.16s
463:	learn: 0.1094137	total: 7.05s	remaining: 8.14s
464:	learn: 0.1094124	total: 7.06s	remaining: 8.12s
465:	learn: 0.1094112	total: 7.07s	remaining: 8.11s
466:	learn: 0.

615:	learn: 0.1093327	total: 9.67s	remaining: 6.03s
616:	learn: 0.1093322	total: 9.69s	remaining: 6.01s
617:	learn: 0.1093316	total: 9.7s	remaining: 6s
618:	learn: 0.1093312	total: 9.71s	remaining: 5.98s
619:	learn: 0.1093308	total: 9.72s	remaining: 5.96s
620:	learn: 0.1093304	total: 9.73s	remaining: 5.94s
621:	learn: 0.1093297	total: 9.75s	remaining: 5.92s
622:	learn: 0.1093288	total: 9.76s	remaining: 5.9s
623:	learn: 0.1093283	total: 9.77s	remaining: 5.88s
624:	learn: 0.1093277	total: 9.78s	remaining: 5.87s
625:	learn: 0.1093273	total: 9.79s	remaining: 5.85s
626:	learn: 0.1093267	total: 9.8s	remaining: 5.83s
627:	learn: 0.1093259	total: 9.81s	remaining: 5.81s
628:	learn: 0.1093254	total: 9.83s	remaining: 5.79s
629:	learn: 0.1093248	total: 9.84s	remaining: 5.78s
630:	learn: 0.1093246	total: 9.85s	remaining: 5.76s
631:	learn: 0.1093241	total: 9.86s	remaining: 5.74s
632:	learn: 0.1093238	total: 9.87s	remaining: 5.72s
633:	learn: 0.1093230	total: 9.88s	remaining: 5.71s
634:	learn: 0.1093

786:	learn: 0.1092569	total: 11.9s	remaining: 3.21s
787:	learn: 0.1092565	total: 11.9s	remaining: 3.2s
788:	learn: 0.1092563	total: 11.9s	remaining: 3.18s
789:	learn: 0.1092556	total: 11.9s	remaining: 3.17s
790:	learn: 0.1092550	total: 11.9s	remaining: 3.15s
791:	learn: 0.1092546	total: 12s	remaining: 3.14s
792:	learn: 0.1092543	total: 12s	remaining: 3.13s
793:	learn: 0.1092538	total: 12s	remaining: 3.11s
794:	learn: 0.1092537	total: 12s	remaining: 3.1s
795:	learn: 0.1092534	total: 12s	remaining: 3.09s
796:	learn: 0.1092532	total: 12.1s	remaining: 3.08s
797:	learn: 0.1092525	total: 12.1s	remaining: 3.07s
798:	learn: 0.1092520	total: 12.1s	remaining: 3.05s
799:	learn: 0.1092515	total: 12.2s	remaining: 3.04s
800:	learn: 0.1092506	total: 12.2s	remaining: 3.02s
801:	learn: 0.1092497	total: 12.2s	remaining: 3s
802:	learn: 0.1092495	total: 12.2s	remaining: 2.99s
803:	learn: 0.1092490	total: 12.2s	remaining: 2.97s
804:	learn: 0.1092487	total: 12.2s	remaining: 2.96s
805:	learn: 0.1092485	total

945:	learn: 0.1091897	total: 13.9s	remaining: 793ms
946:	learn: 0.1091896	total: 13.9s	remaining: 778ms
947:	learn: 0.1091889	total: 13.9s	remaining: 763ms
948:	learn: 0.1091887	total: 13.9s	remaining: 749ms
949:	learn: 0.1091880	total: 13.9s	remaining: 734ms
950:	learn: 0.1091875	total: 14s	remaining: 719ms
951:	learn: 0.1091868	total: 14s	remaining: 704ms
952:	learn: 0.1091864	total: 14s	remaining: 689ms
953:	learn: 0.1091862	total: 14s	remaining: 674ms
954:	learn: 0.1091860	total: 14s	remaining: 659ms
955:	learn: 0.1091858	total: 14s	remaining: 645ms
956:	learn: 0.1091857	total: 14s	remaining: 630ms
957:	learn: 0.1091853	total: 14s	remaining: 615ms
958:	learn: 0.1091846	total: 14s	remaining: 600ms
959:	learn: 0.1091839	total: 14s	remaining: 585ms
960:	learn: 0.1091838	total: 14.1s	remaining: 571ms
961:	learn: 0.1091836	total: 14.1s	remaining: 556ms
962:	learn: 0.1091832	total: 14.1s	remaining: 541ms
963:	learn: 0.1091829	total: 14.1s	remaining: 527ms
964:	learn: 0.1091822	total: 14.

## Ensemble result

In [88]:
ensemble_res = []

for clf, _ in clf_params:
    ensemble_res.append(metas[clf].predict(X_test_stacked))
    ensemble_res.append(tune_results[clf][0].predict(X_test))

In [89]:
import scipy

# make prediction
ensemble_arr = np.array(ensemble_res)
ensemble_pred = scipy.stats.mode(ensemble_arr, axis=0)[0][0,:]

In [90]:
for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = ensemble_pred
    
sample.to_csv('./data/ensemble_sample_p', index=False,float_format='%.5f')

In [91]:
tune_results

{catboost.core.CatBoostRegressor: (<catboost.core.CatBoostRegressor at 0x12341f7b8>,
  {'depth': 3, 'iterations': 5000, 'learning_rate': 0.01},
  0.1097740874764656,
  8.829829331239065),
 lightgbm.sklearn.LGBMRegressor: (LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                depth=3, importance_type='split', learning_rate=0.05,
                max_depth=-1, min_child_samples=20, min_child_weight=0.001,
                min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
                objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
                silent=True, subsample=1.0, subsample_for_bin=200000,
                subsample_freq=0),
  {'depth': 3, 'learning_rate': 0.05},
  0.013457734983486793,
  0.16988393068313598)}

In [ ]:
--

# Stacked model (top 2 models)

In [ ]:
## plot the relationship of the predicted values between Random forest and Catboost
predictions = pd.DataFrame({"lightgbm": gbm_y_pred, "CatBoost": model_cat_y_pred})
predictions.plot(x = "lightg bm", y = "CatBoost", kind = "scatter")

Stacked_preds_result = [a+b for a,b in zip(0.5*gbm_y_pred, 0.5* model_cat_y_pred)]

In [ ]:
for c in sample.columns[sample.columns!='ParcelId']:
    sample[c] = Stacked_preds_result
    
sample.to_csv('./data/Stacked_sample_p', index=False,float_format='%.5f')

# Ensemble sample testing

[Ensemble-stacking](https://www.kaggle.com/wangsg/ensemble-stacking-lb-0-6436)

```python

class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(KFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                y_holdout = y[test_idx]
                print ("Fit Model %d fold %d" % (i, j))
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_holdout)[:]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict(T)[:]
            S_test[:, i] = S_test_i.mean(axis=1)

        # results = cross_val_score(self.stacker, S_train, y, cv=5, scoring='r2')
        # print("Stacker score: %.4f (%.4f)" % (results.mean(), results.std()))
        # exit()

        self.stacker.fit(S_train, y)
        res = self.stacker.predict(S_test)[:]
        return res
    
# model
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# rf params
rf_params = {}
rf_params['n_estimators'] = 50
rf_params['max_depth'] = 8
rf_params['min_samples_split'] = 100
rf_params['min_samples_leaf'] = 30

# xgb params
xgb_params = {}
#xgb_params['n_estimators'] = 50
xgb_params['min_child_weight'] = 12
xgb_params['learning_rate'] = 0.12
xgb_params['max_depth'] = 6
xgb_params['subsample'] = 0.77
xgb_params['reg_lambda'] = 0.8
xgb_params['reg_alpha'] = 0.4
xgb_params['base_score'] = 0
#xgb_params['seed'] = 400
xgb_params['silent'] = 1

# lgb params
lgb_params = {}
lgb_params['n_estimators'] = 450
lgb_params['max_bin'] = 8
lgb_params['learning_rate'] = 0.037 # shrinkage_rate
lgb_params['metric'] = 'l1'          # or 'mae'
lgb_params['sub_feature'] = 0.35    
lgb_params['bagging_fraction'] = 0.85 # sub_row
lgb_params['bagging_freq'] = 40
lgb_params['num_leaves'] = 512        # num_leaf
lgb_params['min_data'] = 500         # min_data_in_leaf
lgb_params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
lgb_params['verbose'] = 0
lgb_params['feature_fraction_seed'] = 2
lgb_params['bagging_seed'] = 3

# XGB model
xgb_model = XGBRegressor(**xgb_params)

# lgb model
lgb_model = LGBMRegressor(**lgb_params)

# RF model
rf_model = RandomForestRegressor(**rf_params)

# ET model
et_model = ExtraTreesRegressor()

# DecsionTree model
dt_model = DecisionTreeRegressor()

# AdaBoost model
ada_model = AdaBoostRegressor()

stack = Ensemble(n_splits=5,
        stacker=LinearRegression(),
        base_models=(rf_model, xgb_model, lgb_model, et_model, ada_model))

y_test_combined = stack.fit_predict(x_train, y_train, x_test) ```